In [124]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import *
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder


In [125]:
def Feature_Encoder(X, cz):
    for c in cz:
        lbl = LabelEncoder()
        lbl.fit(list(X[c].values))
        X[c] = lbl.transform(list(X[c].values))
    return X


In [126]:
weatherData = pd.read_csv("weather.csv")
taxiRidersData = pd.read_csv("taxi-rides.csv")
#taxiRidersData.time_stamp = pd.to_datetime(taxiRidersData.time_stamp,unit="ms")
#weatherData.time_stamp = pd.to_datetime(weatherData.time_stamp,unit='s')


weatherData.rain has 82% null values
groub weatherData by location since it's the common column between two CSVs

In [127]:
#weatherData
weatherData.dropna(axis=1,inplace=True)
avgWeather=weatherData.groupby("location").mean().reset_index(drop=False)

sourceWeather = avgWeather.rename(
    columns={
        'location': 'source',
        'temp': 'source_temp',
        'clouds': 'source_clouds',
        'pressure': 'source_pressure',
        'humidity': 'source_humidity',
        'wind': 'source_wind'
    }
)

destinationWeather = avgWeather.rename(
    columns={
        'location': 'destination',
        'temp': 'destination_temp',
        'clouds': 'destination_clouds',
        'pressure': 'destination_pressure',
        'humidity': 'destination_humidity',
        'wind': 'destination_wind'
    }
)


In [128]:
data = taxiRidersData.merge(sourceWeather,on='source').merge(destinationWeather,on='destination')

In [129]:
X = data.drop(['price'],axis=1)
Y = data['price']

In [130]:
X.time_stamp = pd.to_datetime(X.time_stamp,unit="s")
X=X.drop(['id'],axis=1)
X.head()

,distance,cab_type,time_stamp_x,destination,source,surge_multiplier,product_id,name,source_temp,source_clouds,source_pressure,time_stamp_y,source_humidity,source_wind,destination_temp,destination_clouds,destination_pressure,time_stamp,destination_humidity,destination_wind
0,0.62,Uber,1.544770e+12,West End,Haymarket Square,1.0,8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a,Taxi,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
1,0.64,Lyft,1.543420e+12,West End,Haymarket Square,1.0,lyft_premier,Lux,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
2,0.72,Lyft,1.544820e+12,West End,Haymarket Square,1.0,lyft_line,Shared,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
3,0.65,Lyft,1.543400e+12,West End,Haymarket Square,1.0,lyft_luxsuv,Lux Black XL,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
4,0.61,Uber,1.544960e+12,West End,Haymarket Square,1.0,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233


since each name has it's product_id we can drop one of them since it's considered dublication.


Use binary Encoding on "cab_type" since it only has two uniuqe features (Uber,Lyft)

In [131]:
temp = X.product_id +" "+ X.name
print(temp.unique())

['8cf7e821-f0d3-49c6-8eba-e679c0ebcf6a Taxi' 'lyft_premier Lux'
 'lyft_line Shared' 'lyft_luxsuv Lux Black XL'
 '55c66225-fbe7-4fd5-9072-eab1ece5e23e UberX'
 '9a0e7b09-b92b-4c41-9779-2ad22b4d779d WAV' 'lyft_lux Lux Black'
 '6f72dfc5-27f1-42e8-84db-ccc7a75f6969 UberXL'
 '997acbb5-e102-41e1-b155-9df7de0a73f2 UberPool' 'lyft_plus Lyft XL'
 '6c84fd89-3f11-4782-9b50-97c468b19529 Black'
 '6d318bcc-22a3-4af6-bddd-b409bfce1546 Black SUV' 'lyft Lyft']


In [132]:
X=X.drop(['product_id'],axis=1)
X.cab_type

0         Uber
1         Lyft
2         Lyft
3         Lyft
4         Uber
          ... 
554451    Lyft
554452    Uber
554453    Uber
554454    Uber
554455    Uber
Name: cab_type, Length: 554456, dtype: object

In [133]:
di = {"Uber":1,"Lyft":0}
X.cab_type.replace(di,inplace=True)


In [134]:
X = Feature_Encoder(X,["destination","source","name"])
X.destination

0         11
1         11
2         11
3         11
4         11
          ..
554451     5
554452     5
554453     5
554454     5
554455     5
Name: destination, Length: 554456, dtype: int64

In [135]:
X.head(10)

,distance,cab_type,time_stamp_x,destination,source,surge_multiplier,name,source_temp,source_clouds,source_pressure,time_stamp_y,source_humidity,source_wind,destination_temp,destination_clouds,destination_pressure,time_stamp,destination_humidity,destination_wind
0,0.62,1,1.544770e+12,11,5,1.0,8,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
1,0.64,0,1.543420e+12,11,5,1.0,2,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
2,0.72,0,1.544820e+12,11,5,1.0,7,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
3,0.65,0,1.543400e+12,11,5,1.0,4,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
4,0.61,1,1.544960e+12,11,5,1.0,10,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
5,0.62,1,1.544890e+12,11,5,1.0,12,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
6,0.61,1,1.544860e+12,11,5,1.0,10,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
7,0.72,1,1.543610e+12,11,5,1.0,8,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
8,0.68,0,1.545140e+12,11,5,1.0,3,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233
9,0.66,0,1.543330e+12,11,5,1.0,4,39.067897,0.676711,1008.445239,1.543857e+09,0.764837,6.843193,38.983403,0.677247,1008.44109,2018-12-03 17:16:17.214149120,0.767266,6.816233


In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20,shuffle=True,random_state=10)